In [1]:
from Continuo.ContinuoMarkov import scale_degree_harmonization, scale_degree_harmonization_song, scale_degree_harmonization_untransposed, markov_cluster_data
from Continuo.kmeans_inversion import k_means_data, silhouette_analysis
from alfabeto_sources.all_sources import *

from scipy.spatial.distance import euclidean
import numpy as np
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph

import csv

In [2]:
def figure_sorter(function):
    final_data = []
    other_numbers = []
    figures = [[0, 4, 7], [0, 3, 7], [0, 3, 8], [0, 4, 9], [0, 5, 9], [0, 5, 8]]
    for figure in figures:
        figure_list = []
        for number in range(12):
            fig = str([number, figure])
            if fig in function:
                figure_list.append(function[fig])
            else:
#                 figure_list.append('-')
                figure_list.append(0)
        final_data.append(figure_list)
    for number in range(12):
        numbers = 0
        for x, y in function.items():
            if x[1:3] == str(number) and x[-10:-1] not in str(figures):
                numbers += y
            elif x[1:2] == str(number) and x[-10:-1] not in str(figures):
                numbers += y
        other_numbers.append(numbers)
    other_final = []
    for x in other_numbers:
        if x > 0:
            other_final.append(x)
        else:
            other_final.append('-')
#     final_data.append(other_final) # if you want dashes instead of 0's #---none of these if no other desired
    final_data.append(other_numbers) # if you want 0's instead of dashes
    return final_data

In [ ]:
def inversion_kmeans(corpus, mode_list):
    all_modes = []
    if mode_list == 'all':
        each_mode = [('f', 0), ('f',2), ('f',5), ('f',7), ('f',9), ('f',10), 
                     ('n',0), ('n',2), ('n',4), ('n',5), ('n',7), ('n',9)]
        for x in each_mode:
            all_modes.append(x)
    else:
        for x in mode_list:
            all_modes.append(x)
    labels = []
    final_labels = []
    final_songs = []
    final_songs_reduced = []
    for x in all_modes:
        for y in corpus:
            for z in y.values():
                if z['data']['key'] == x[0] and z['data']['final'] == x[1]:
                    temp_list = []
                    matrix_data = (figure_sorter(scale_degree_harmonization_song(z, x)))
                    for a in matrix_data:
                        for b in a:
                            temp_list.append(b)
                    final_songs.append(temp_list)
                    labels.append(x)
    for x in labels:
        if x == ('f',2):
            final_labels.append('$♭:D$')
        elif x == ('n',4):
            final_labels.append('$♮:E$')
        elif x == ('n',9):
            final_labels.append('$♮:A$')
        elif x == ('f',7):
            final_labels.append('$♭:G$')
        elif x == ('n',2):
            final_labels.append('$♮:D$')
        elif x == ('f',9):
            final_labels.append('$♭:A$')
        elif x == ('f',0):
            final_labels.append('$♭:C$')
        elif x == ('f',10):
            final_labels.append('$♭:B♭$')
        elif x == ('n',5):
            final_labels.append('$♮:F$')
        elif x == ('n',0):
            final_labels.append('$♮:C$')
        elif x == ('f',5):
            final_labels.append('$♭:F$')
        elif x == ('n',7):
            final_labels.append('$♮:G$')
        elif x == 'Temperley MAJOR':
            final_labels.append('$Temperley MAJOR$')
        elif x == 'Temperley MINOR':
            final_labels.append('$Temperley MINOR$')
        else:
            final_labels.append('o') 
    for x in final_songs:
        for y in x:
            final_songs_reduced.append(y)
    
    return (final_songs, final_labels)

def labeler(label_data):
    final_labels = []
    for x in label_data:
        if x == ('f',2):
            final_labels.append('$♭:D$')
        elif x == ('n',4):
            final_labels.append('$♮:E$')
        elif x == ('n',9):
            final_labels.append('$♮:A$')
        elif x == ('f',7):
            final_labels.append('$♭:G$')
        elif x == ('n',2):
            final_labels.append('$♮:D$')
        elif x == ('f',9):
            final_labels.append('$♭:A$')
        elif x == ('f',0):
            final_labels.append('$♭:C$')
        elif x == ('f',10):
            final_labels.append('$♭:B♭$')
        elif x == ('n',5):
            final_labels.append('$♮:F$')
        elif x == ('n',0):
            final_labels.append('$♮:C$')
        elif x == ('f',5):
            final_labels.append('$♭:F$')
        elif x == ('n',7):
            final_labels.append('$♮:G$')
        elif x == 'Temperley MAJOR':
            final_labels.append('$Temperley MAJOR$')
        elif x == 'Temperley MINOR':
            final_labels.append('$Temperley MINOR$')
        else:
            final_labels.append('o') 
    return final_labels

In [ ]:
alf_data = inversion_kmeans(GetAll.all_alf, 'all')

In [ ]:
def averaged_octave_harmonization(flattened_list_with_labels):
    total_chunks = []
    averaged_chunks = []
    all_keys = []
    final_unchunked = []
    for x in flattened_list_with_labels[1]:
        if x not in all_keys:
            all_keys.append(x)
    for x in all_keys:
        a_key = []
        for a, b in zip(flattened_list_with_labels[0], flattened_list_with_labels[1]):
            if b == x:
                a_key.append(a)
        averaged_chunks.append([sum(col) / len(col) for col in zip(*a_key)])
    
    for instance in averaged_chunks:
        total_chunks.append([instance[x:x+12] for x in range(0, len(instance), 12)])
#     for x in total_chunks:
#         for i, (a, b, c, d, e, f, g) in enumerate(zip(x[0], x[1], x[2], x[3], x[4], x[5], x[6])):
#             maxy = max(a, b, c, d, e, f, g)
#             if a == maxy and maxy > 0:
#                 x[0][i] = 1
#             else:
#                 x[0][i] = 0
#             if b == maxy and maxy > 0:
#                 x[1][i] = 1
#             else:
#                 x[1][i] = 0
#             if c == maxy and maxy > 0:
#                 x[2][i] = 1
#             else:
#                 x[2][i] = 0
#             if d == maxy and maxy > 0:
#                 x[3][i] = 1
#             else:
#                 x[3][i] = 0
#             if e == maxy and maxy > 0:
#                 x[4][i] = 1
#             else:
#                 x[4][i] = 0
#             if f == maxy and maxy > 0:
#                 x[5][i] = 1
#             else:
#                 x[5][i] = 0
#             if g == maxy and maxy > 0:
#                 x[6][i] = 1
#             else:
#                 x[6][i] = 0
    for x in total_chunks:
        song_temp = []
        for y in x:
            for z in y:
                song_temp.append(z)
        final_unchunked.append(song_temp)
    return final_unchunked, all_keys

In [ ]:
XY = averaged_octave_harmonization(alf_data)

In [ ]:
len(XY[1])

In [ ]:
%matplotlib inline

for x in range(2, 13):
    k_means_data(XY[0], x, XY[1])


In [ ]:
# inversion_data = inversion_kmeans(GetAll.all_alf, [('n', 2), ('f', 2)])

all_books = []
for x in range(5):
    all_books.append(inversion_kmeans([GetComposer.all_milanuzzi[x]],  'all'))
# inversion_data = inversion_kmeans(GetComposer.all_kaps[0:3],  'all')
early_kap = inversion_kmeans(GetComposer.all_kaps[0:3],  'all')
late_kap = inversion_kmeans(GetComposer.all_kaps[3:6],  'all')

In [ ]:
for x in all_books:
    print(k_means_data(x[0], 2, x[1]))

In [ ]:
%matplotlib inline
# PSA = k_means_data(inversion_data[0], 3, inversion_data[1])
k_means_data(early_kap[0], 2, early_kap[1])
k_means_data(late_kap[0], 2, late_kap[1])

# k_means_data(inversion_data[0], 3, inversion_data[1])
# k_means_data(inversion_data[0], 12, inversion_data[1])

In [ ]:
for x in range(2, 15):
    print(x, 'yields', k_means_data(inversion_data[0], x, inversion_data[1])[1])

In [ ]:
%matplotlib inline
silhouette_analysis(PSA, 6)

In [ ]:
def neato_graph(list_of_lists, label_lists):
    dt = [('len', float)]

    A = (np.array(list_of_lists)*5).view(dt)

    G = nx.from_numpy_matrix(A)
    relabeled_nodes = {}
    for x, y in zip (range(len(label_lists)), label_lists):
        relabeled_nodes[x] = str((x, y[1:-1]))
    G = nx.relabel_nodes(G, relabeled_nodes)

    G = to_agraph(G)

    G.node_attr.update(color="green", style="filled", fontsize=30)
    G.edge_attr.update(color="black", width=0.5)
#     G.edge_attr.update(color='black', style='invis', width=0)
    # G.graph_attr.update(landscape='true', ranksep='0.1')

    G.draw('/home/daniel/Desktop/neato_ave.dot', format='dot', prog='neato')
    G.draw

In [ ]:
inversion_data = inversion_kmeans(GetComposer.all_kaps,  'all')

In [ ]:
DIST = []
for x in XY[0]:
    dist_temp = []
    for y in XY[0]:
        dist_temp.append(euclidean(x, y))
    DIST.append(dist_temp)

In [ ]:
neato_graph(DIST, XY[1])

In [7]:
all_flats = [('f', 5), ('f', 0), ('f', 10), ('f', 2), ('f', 7)]
all_naturals = [('n', 0), ('n', 7), ('n', 5), ('n', 2), ('n', 9), ('n', 4)]

In [16]:
CSV_DATA = figure_sorter(scale_degree_harmonization(
        GetAll.all_alf, ('n', 7)))

In [17]:
with open('/home/daniel/Desktop/n7.csv', 'w', newline='') as fp:
    a = csv.writer(fp, delimiter='&')
    final_data = CSV_DATA
    a.writerows(final_data)

In [ ]:
a = '[0, [2, 7, 0]]'

In [ ]:
b = [0, [2, 7, 0]]


In [ ]:
'0' in str(b)

In [ ]:
pre_20 = inversion_kmeans([GetComposer.all_kaps[0], GetComposer.all_montesardo[0], GetComposer.all_giaccio[0], 
                           GetComposer.all_giaccio[1], GetComposer.all_kaps[1], GetComposer.all_kaps[2], 
                           GetComposer.all_rontani[0]],  'all')
# early_20 = inversion_kmeans([GetComposer.all_milanuzzi[x]],  'all')
# late_20 = inversion_kmeans([GetComposer.all_milanuzzi[x]],  'all')
post_20 = inversion_kmeans([GetComposer.all_milanuzzi[3], GetComposer.all_kaps[4], GetComposer.all_kaps[5], 
                           GetComposer.all_milanuzzi[4], GetComposer.all_kaps[6]],  'all')

In [ ]:
k_means_data(pre_20[0], 2, pre_20[1])
k_means_data(post_20[0], 2, post_20[1])